In [1]:
# Import libraries
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime, timedelta
import yfinance as yf
import time  # For retries
import logging  # Optional: For better logging

# Set up logging (replace print with logging for production)
logging.basicConfig(level=logging.INFO)

print("Libraries imported successfully!")  # Keeping print for consistency with original
print(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("Using Plotly for interactive visualizations! 📊")

# Stock Tickers Configuration
STOCK_TICKERS = []

# Use realistic past dates for actual data
BASELINE_DATE = '2000-01-01'  # Changed to past date that exists

# Historical Comparison - Compare performance to current date
future_date_str = datetime.now().strftime('%Y-%m-%d')  # Use current date for comparison

# Will store baseline prices after fetching

baseline_prices = {}

print("✅ Stock tickers configured successfully!")
print(f"📅 Baseline date: {BASELINE_DATE}")
print(f"📈 Tracking {len(STOCK_TICKERS)} stocks")

class StockTracker:
    def __init__(self, tickers: list[str], baseline_date: str):
        self.tickers = tickers
        self.baseline_date = baseline_date
        self.baseline_prices = self.fetch_stock_prices(baseline_date)

    def fetch_stock_prices(self, date_str: str | None = None, compare_to_baseline: bool = False) -> dict[str, float]:
        """
        Fetch current or historical prices for all stocks in self.tickers using batch download
        
        Parameters:
        - date_str: Optional date string in 'YYYY-MM-DD' format. If None, gets latest prices.
        - compare_to_baseline: If True, shows percentage change from baseline
        
        Returns:
        - dict: {ticker: price}
        """
        if date_str is None:
            date_str = datetime.now().strftime('%Y-%m-%d')
            print(f"📈 Fetching LATEST stock prices (around {date_str})")
        else:
            print(f"📈 Fetching stock prices for: {date_str}")
        
        print("=" * 60)
        
        fetched_prices = {}
        try:
            start_date = datetime.strptime(date_str, '%Y-%m-%d') - timedelta(days=7)  # Buffer for non-trading days
            end_date = datetime.strptime(date_str, '%Y-%m-%d') + timedelta(days=1)
            
            for attempt in range(3):  # Retry up to 3 times
                data = yf.download(self.tickers, start=start_date, end=end_date)['Close']
                if not data.empty:
                    break
                print(f"Retry {attempt + 1}/3: No data, waiting...")
                time.sleep(2)
            
            if data.empty:
                print("❌ No data available for the specified period.")
                return {}
            
            successful_fetches = 0
            for ticker in self.tickers:
                if ticker in data.columns and not data[ticker].dropna().empty:
                    price = round(data[ticker].dropna().iloc[-1], 2)  # Latest available close
                    fetched_prices[ticker] = price
                    if compare_to_baseline and self.baseline_prices and ticker in self.baseline_prices:
                        baseline_price = self.baseline_prices[ticker]
                        if baseline_price is not None:
                            pct_change = ((price - baseline_price) / baseline_price) * 100
                            print(f"🔍 {ticker}: ${price:.2f} ({pct_change:+.1f}%)")
                        else:
                            print(f"🔍 {ticker}: ${price:.2f} (No baseline)")
                    else:
                        print(f"🔍 {ticker}: ${price:.2f}")
                    successful_fetches += 1
                else:
                    print(f"🔍 {ticker}: Failed")
            
            print(f"\n✅ Successfully fetched prices for {successful_fetches}/{len(self.tickers)} stocks")
            return fetched_prices
        except Exception as e:
            print(f"❌ Error in batch fetch: {e}")
            return {}

    def track_stocks(self, target_date: str, current_prices: dict[str, float]) -> pd.DataFrame | None:
        """
        Track stock performance from baseline to target date
        
        Parameters:
        - target_date: String (e.g., "2025-08-01")
        - current_prices: Dictionary with stock symbols and current prices
        
        Returns:
        - pd.DataFrame: Performance data, or None if no data
        """
        results = []
        
        print(f"📈 STOCK PERFORMANCE ANALYSIS")
        print(f"From: {self.baseline_date} → To: {target_date}")
        print("=" * 50)
        
        for symbol, current_price in current_prices.items():
            if symbol in self.baseline_prices and current_price is not None:
                baseline_price = self.baseline_prices[symbol]
                if baseline_price is not None:
                    price_change = current_price - baseline_price
                    pct_change = (price_change / baseline_price) * 100
                    
                    results.append({
                        'Symbol': symbol,
                        'Baseline': baseline_price,
                        'Current': current_price,
                        'Change': price_change,
                        'Percent_Change': pct_change
                    })
        
        if not results:
            print("No valid price data provided.")
            return None
        
        # Create DataFrame and sort by performance
        df = pd.DataFrame(results)
        df = df.sort_values('Percent_Change', ascending=False)
        
        # Display results
        print(f"{'Rank':<4} {'Symbol':<6} {'Baseline':<10} {'Current':<10} {'Change':<10} {'% Change':<10}")
        print("-" * 60)
        
        for i, (_, row) in enumerate(df.iterrows(), 1):
            print(f"{i:<4} {row['Symbol']:<6} ${row['Baseline']:<9.2f} ${row['Current']:<9.2f} "
                  f"${row['Change']:<9.2f} {row['Percent_Change']:<9.2f}%")
        
        # Summary
        avg_change = df['Percent_Change'].mean()
        print(f"\n🏆 Best Performer: {df.iloc[0]['Symbol']} ({df.iloc[0]['Percent_Change']:+.2f}%)")
        print(f"📉 Worst Performer: {df.iloc[-1]['Symbol']} ({df.iloc[-1]['Percent_Change']:+.2f}%)")
        print(f"📊 Average Change: {avg_change:+.2f}%")
        
        return df

    def create_performance_plot(self, target_date: str, current_prices: dict[str, float]) -> pd.DataFrame | None:
        """
        Create interactive visualization for stock performance using Plotly
        
        Parameters:
        - target_date: String (e.g., "2025-08-01")
        - current_prices: Dictionary with stock symbols and current prices
        
        Returns:
        - pd.DataFrame: Performance data, or None if no data
        """
        df = self.track_stocks(target_date, current_prices)
        
        if df is None:
            return None
        
        # Sort x-axis by performance
        df = df.sort_values('Percent_Change', ascending=False)
        
        # Calculate average change
        avg_change = df['Percent_Change'].mean()
        
        # Create color scheme based on performance
        colors = ['#00CC44' if x > 0 else '#FF4444' for x in df['Percent_Change']]
        
        # Create interactive bar chart
        fig = go.Figure()
        
        fig.add_trace(go.Bar(
            x=df['Symbol'],
            y=df['Percent_Change'],
            marker_color=colors,
            text=[f'{x:.1f}%' for x in df['Percent_Change']],
            textposition='outside',
            hovertemplate=
            '<b>%{x}</b><br>' +
            'Baseline: $%{customdata[0]:.2f}<br>' +
            'Current: $%{customdata[1]:.2f}<br>' +
            'Change: %{y:.2f}%<br>' +
            '<extra></extra>',
            customdata=list(zip(df['Baseline'], df['Current'])),
            showlegend=False
        ))
        
        # Add zero line
        fig.add_hline(y=0, line_dash="dash", line_color="black", opacity=0.5)
        
        # Add average change line
        fig.add_hline(
            y=avg_change,
            line_dash="dash",
            line_color="blue",
            annotation_text=f"Avg Change: {avg_change:.2f}%",
            annotation_position="bottom right"
        )
        
        # Add annotations for best and worst
        fig.add_annotation(x=df['Symbol'].iloc[0], y=df['Percent_Change'].iloc[0] + 5,
                           text="Best", showarrow=True, arrowhead=1)
        fig.add_annotation(x=df['Symbol'].iloc[-1], y=df['Percent_Change'].iloc[-1] - 5,
                           text="Worst", showarrow=True, arrowhead=1)
        
        # Update layout
        fig.update_layout(
            title={
                'text': f'📈 Stock Performance: {self.baseline_date} → {target_date}',
                'x': 0.5,
                'font': {'size': 20, 'family': 'Arial Black'}
            },
            xaxis_title="Stock Symbol",
            yaxis_title="Percentage Change (%)",
            height=600,
            width=1000,
            template="plotly_white",
            font=dict(size=12),
            hovermode='x',
            plot_bgcolor='rgba(240, 240, 240, 0.1)'
        )
        
        # Update axes
        fig.update_xaxes(tickangle=45, tickfont=dict(size=11), categoryorder='array', categoryarray=df['Symbol'].tolist())
        fig.update_yaxes(tickfont=dict(size=11), gridcolor='rgba(128, 128, 128, 0.3)')
        
        # Show the interactive plot
        fig.show()
        
        # Optional: Export
        # fig.write_html('performance.html')
        # fig.write_image('performance.png')
        
        return df


def run_trade_return(STOCK_TICKERS, BASELINE_DATE, future_date_str):
    """
    Run complete stock trading analysis from baseline to target date
    
    Parameters:
    - STOCK_TICKERS: List of stock ticker symbols
    - BASELINE_DATE: Baseline date string (YYYY-MM-DD)
    - future_date_str: Target date string (YYYY-MM-DD)
    
    Returns:
    - pd.DataFrame: Performance analysis results
    """
    # Initialize tracker
    tracker = StockTracker(STOCK_TICKERS, BASELINE_DATE)

    print(f"\n📊 Baseline Prices ({BASELINE_DATE}):")
    print(tracker.baseline_prices)

    print("✅ Stock tracking class ready!")
    print("📊 Interactive Plotly visualization ready!")

    print(f"📊 HISTORICAL COMPARISON")
    print(f"Baseline: {BASELINE_DATE} → Target: {future_date_str}")
    print("=" * 50)

    # Fetch prices for future date
    future_prices = tracker.fetch_stock_prices(future_date_str, compare_to_baseline=True)

    # DEMO: Interactive Plotly Visualizations
    print("\n🎯 Historical Performance Analysis")
    print("=" * 45)

    print("Creating interactive Plotly charts...")
    print(f"\nPerformance from {BASELINE_DATE} to {future_date_str}:")
    demo_result = tracker.create_performance_plot(future_date_str, future_prices)

    return demo_result

Libraries imported successfully!
Date: 2025-09-25 21:38:43
Using Plotly for interactive visualizations! 📊
✅ Stock tickers configured successfully!
📅 Baseline date: 2000-01-01
📈 Tracking 0 stocks


In [2]:
# Use realistic past dates for actual data
BASELINE_DATE = '2025-09-21' 

# Historical Comparison - Compare performance to current date
future_date_str = '2025-09-25'

In [3]:
# ChatGPT
STOCK_TICKERS = [
    'MU',    # Micron Technology
    'TSLA',  # Tesla, Inc.
    'APLD',  # Applied Digital Corp
    'CRNX',  # Crinetics Pharmaceuticals
    'SNOW',  # Snowflake Inc.
]

result = run_trade_return(STOCK_TICKERS, BASELINE_DATE, future_date_str)

📈 Fetching stock prices for: 2025-09-21
YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  5 of 5 completed


🔍 MU: $162.73
🔍 TSLA: $426.07
🔍 APLD: $20.48
🔍 CRNX: $33.91
🔍 SNOW: $230.48

✅ Successfully fetched prices for 5/5 stocks

📊 Baseline Prices (2025-09-21):
{'MU': 162.73, 'TSLA': 426.07, 'APLD': 20.48, 'CRNX': 33.91, 'SNOW': 230.48}
✅ Stock tracking class ready!
📊 Interactive Plotly visualization ready!
📊 HISTORICAL COMPARISON
Baseline: 2025-09-21 → Target: 2025-09-25
📈 Fetching stock prices for: 2025-09-25


[*********************100%***********************]  5 of 5 completed


🔍 MU: $155.84 (-4.2%)
🔍 TSLA: $427.14 (+0.3%)
🔍 APLD: $21.50 (+5.0%)
🔍 CRNX: $34.52 (+1.8%)
🔍 SNOW: $220.12 (-4.5%)

✅ Successfully fetched prices for 5/5 stocks

🎯 Historical Performance Analysis
Creating interactive Plotly charts...

Performance from 2025-09-21 to 2025-09-25:
📈 STOCK PERFORMANCE ANALYSIS
From: 2025-09-21 → To: 2025-09-25
Rank Symbol Baseline   Current    Change     % Change  
------------------------------------------------------------
1    APLD   $20.48     $21.50     $1.02      4.98     %
2    CRNX   $33.91     $34.52     $0.61      1.80     %
3    TSLA   $426.07    $427.14    $1.07      0.25     %
4    MU     $162.73    $155.84    $-6.89     -4.23    %
5    SNOW   $230.48    $220.12    $-10.36    -4.49    %

🏆 Best Performer: APLD (+4.98%)
📉 Worst Performer: SNOW (-4.49%)
📊 Average Change: -0.34%


In [4]:
# grok
STOCK_TICKERS = [
    'NVDA',  # NVIDIA Corporation
    'TSLA',  # Tesla Inc
    'PLTR',  # Palantir Technologies Inc
    'MU',    # Micron Technology Inc
    'CRWD',  # CrowdStrike Holdings Inc
]

result = run_trade_return(STOCK_TICKERS, BASELINE_DATE, future_date_str)

[*******************   40%                       ]  2 of 5 completed

📈 Fetching stock prices for: 2025-09-21


[*********************100%***********************]  5 of 5 completed


🔍 NVDA: $176.67
🔍 TSLA: $426.07
🔍 PLTR: $182.39
🔍 MU: $162.73
🔍 CRWD: $502.55

✅ Successfully fetched prices for 5/5 stocks

📊 Baseline Prices (2025-09-21):
{'NVDA': 176.67, 'TSLA': 426.07, 'PLTR': 182.39, 'MU': 162.73, 'CRWD': 502.55}
✅ Stock tracking class ready!
📊 Interactive Plotly visualization ready!
📊 HISTORICAL COMPARISON
Baseline: 2025-09-21 → Target: 2025-09-25
📈 Fetching stock prices for: 2025-09-25


[*********************100%***********************]  5 of 5 completed

🔍 NVDA: $173.89 (-1.6%)
🔍 TSLA: $427.56 (+0.3%)
🔍 PLTR: $175.31 (-3.9%)
🔍 MU: $155.99 (-4.1%)
🔍 CRWD: $474.99 (-5.5%)

✅ Successfully fetched prices for 5/5 stocks

🎯 Historical Performance Analysis
Creating interactive Plotly charts...

Performance from 2025-09-21 to 2025-09-25:
📈 STOCK PERFORMANCE ANALYSIS
From: 2025-09-21 → To: 2025-09-25
Rank Symbol Baseline   Current    Change     % Change  
------------------------------------------------------------
1    TSLA   $426.07    $427.56    $1.49      0.35     %
2    NVDA   $176.67    $173.89    $-2.78     -1.57    %
3    PLTR   $182.39    $175.31    $-7.08     -3.88    %
4    MU     $162.73    $155.99    $-6.74     -4.14    %
5    CRWD   $502.55    $474.99    $-27.56    -5.48    %

🏆 Best Performer: TSLA (+0.35%)
📉 Worst Performer: CRWD (-5.48%)
📊 Average Change: -2.95%


In [5]:
# claude
STOCK_TICKERS = [
    'MTSR',  # Metsera Inc.
    'QUBT',  # Quantum Computing Inc.
    'MU',    # Micron Technology
    'PONY',  # Pony AI
    'RGTI',  # Rigetti Computing
]

result = run_trade_return(STOCK_TICKERS, BASELINE_DATE, future_date_str)

[                       0%                       ]

📈 Fetching stock prices for: 2025-09-21


[*********************100%***********************]  5 of 5 completed


🔍 MTSR: $33.32
🔍 QUBT: $23.27
🔍 MU: $162.73
🔍 PONY: $20.71
🔍 RGTI: $28.52

✅ Successfully fetched prices for 5/5 stocks

📊 Baseline Prices (2025-09-21):
{'MTSR': 33.32, 'QUBT': 23.27, 'MU': 162.73, 'PONY': 20.71, 'RGTI': 28.52}
✅ Stock tracking class ready!
📊 Interactive Plotly visualization ready!
📊 HISTORICAL COMPARISON
Baseline: 2025-09-21 → Target: 2025-09-25
📈 Fetching stock prices for: 2025-09-25


[*********************100%***********************]  5 of 5 completed

🔍 MTSR: $52.51 (+57.6%)
🔍 QUBT: $20.45 (-12.1%)
🔍 MU: $155.90 (-4.2%)
🔍 PONY: $20.92 (+1.0%)
🔍 RGTI: $29.72 (+4.2%)

✅ Successfully fetched prices for 5/5 stocks

🎯 Historical Performance Analysis
Creating interactive Plotly charts...

Performance from 2025-09-21 to 2025-09-25:
📈 STOCK PERFORMANCE ANALYSIS
From: 2025-09-21 → To: 2025-09-25
Rank Symbol Baseline   Current    Change     % Change  
------------------------------------------------------------
1    MTSR   $33.32     $52.51     $19.19     57.59    %
2    RGTI   $28.52     $29.72     $1.20      4.21     %
3    PONY   $20.71     $20.92     $0.21      1.01     %
4    MU     $162.73    $155.90    $-6.83     -4.20    %
5    QUBT   $23.27     $20.45     $-2.82     -12.12   %

🏆 Best Performer: MTSR (+57.59%)
📉 Worst Performer: QUBT (-12.12%)
📊 Average Change: +9.30%
